In [ ]:
%run -n main.py
USER = input('login')
PASS = get_pass('pass')
AUTH = (USER, PASS)

In [ ]:
q = gh_q(
    '"from yargy" OR "import yargy" OR "from natasha" OR "import natasha"',
    extensions=('py', 'ipynb'),
)
url = gh_search_code_url(q)
response, data = call_gh(url, AUTH, TEXT_MATCH)
total = parse_serp_total(data)
print('Total:', total)

In [ ]:
%%time
serp = []
pages = list(get_pages(total))
while pages:
    page = pages.pop(0)
    print('Page #:', page, 'To do:', len(pages))
    url = gh_search_code_url(q, page=page)
    response, data = call_gh(url, AUTH, headers=TEXT_MATCH)
    if is_broken(data):
        print('Ban')
        pages.insert(0, page)
        sleep(60)
    else:
        serp.extend(parse_serp(data))
        sleep(5)

In [ ]:
cache = set(load_lines(URLS))
urls = {get_serp_record_url(_) for _ in serp}
print('In cache:', len(cache & urls), 'New:', len(urls - cache))

In [ ]:
pattern = '(yargy|natasha)'
for record in serp:
    url = get_serp_record_url(record)
    if url in cache:
        continue
    print(url)
    for text in record.matches:
        spans = get_spans(text, pattern)
        show_markup(text, spans)

In [ ]:
dump_lines(sorted(cache | urls), URLS)